In [1]:
!nvidia-smi

Sat Mar 23 09:29:09 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.147.05   Driver Version: 525.147.05   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:08:00.0  On |                  N/A |
|  0%   48C    P8    21W / 240W |    710MiB /  8192MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import torch
import esm
from Bio import SeqIO



In [3]:
torch.cuda.empty_cache()
torch.cuda.set_per_process_memory_fraction(0.2, 0)
torch.cuda.is_available()
#device = torch.device("cuda:0")
torch.set_num_threads(6)

In [4]:
a = torch.ones(1, device="cuda:0")

In [5]:
!nvidia-smi

Sat Mar 23 09:29:10 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.147.05   Driver Version: 525.147.05   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:08:00.0  On |                  N/A |
|  0%   49C    P2    27W / 240W |    866MiB /  8192MiB |     55%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [6]:
del a
torch.cuda.empty_cache()

In [7]:
with open("./Human_ACE_orthologues.fa") as file:
    fasta_seqs = list(SeqIO.parse(file, "fasta"))

In [8]:
data = [(i.id, str(i.seq)) for i in fasta_seqs]

In [9]:
data[0:1]

[('ENSOGAP00000006131_Bushbaby',
  'MGASSGLRASGPPPVLSLLPLLQLLLLPPSPAAQALDPGLLPGNFSADEAGAQLFAQSYNSSAEQVLFQSTAASWAYSTNITEENAQRQEEAVLLNQRFAEAWGQKAKELYGPIWQNFTDPKLRKVIRAVCTLGPANLPLAKQQQYVSLQTNMSRIYSTTKVCFPNKTATCWSLDPELTNILASSRSYARLLFAWEGWHDTVGIPLKALYQDFTTISNEAYRQDGFSDTGAYWRSWYNSATFEEDLEHLYHQLEPLYLNLHAYVRRALHRRYGDRYINLRGPIPAHLLGDMWAQSWDNLYDMVVPFPGKPNLDVTSTMVKQGWNATHMFRVAEEFFTSLGLSPMPPEFWAESMLEKPADGREVVCHASAWDFYNRKDFRIKQCTQVTMDQLSTVHHEMGHVQYYLQYKDQPVSLREGANPGFHEAIGDVLGLSVSTPAHLHKIGLLDHVTNDTESDINYLLKMALDKIAFLPFGYLVDQWRWGVFSGHTPPSRYNSDWWYLRTKYQGICPPVVRNETHFDAGAKFHIPNGTPYIRYFVSFILQFQFHQALCKEAGHQGPLHQCDIYKSTQAGAKLQEVLRAGSSRPWQEVLKNMTGSDALDAQPLLDYFQPVSQWLQEQNQQNNEILGWPEYQWRPPLPTNYPEGIDLITDEAEANKFVEEYDQVSQVVWNEFAEANWNYNTNITTEASQILLQKNLEIANHTLKWGIQARQFDVSTFQNTTTKRVIKKVQDLDRAALPAKELEEYNKILLEMETTYSVATVCHTNGTCLQLEPDLTSMMATSRQYYELLWAWKSWRDKVGRAILPSFPKYVELTNKAARLNGYIDGGDSWRSMYEMPSLEQNLEELFQELQPLYLNLHAYVRRALHRHYGAQHINLEGPIPAHLLGNMWAQTWSNIYDLVVPFPSAPSMDATEAMIKQGWTPRRMFKEADNFFISLGLLPVPPEFWNKSMLEKPTDGREVVCH

In [10]:
# Load ESM-2 model
model, alphabet = esm.pretrained.esm2_t33_650M_UR50D()
batch_converter = alphabet.get_batch_converter()
model.eval()  # disables dropout for deterministic results
#model = model.to(device)

# Prepare data (first 2 sequences from ESMStructuralSplitDataset superfamily / 4)
#data = [
#    ("protein1", "MKTVRQERLKSIVRILERSKEPVSGAQLAEELSVSRQVIVQDIAYLRSLGYNIVATPRGYVLAGG"),
#    ("protein2", "KALTARQQEVFDLIRDHISQTGMPPTRAEIAQRLGFRSPNAAEEHLKALARKGVIEIVSGASRGIRLLQEE"),
#    ("protein2 with mask","KALTARQQEVFDLIRD<mask>ISQTGMPPTRAEIAQRLGFRSPNAAEEHLKALARKGVIEIVSGASRGIRLLQEE"),
#    ("protein3",  "K A <mask> I S Q"),
#]
batch_labels, batch_strs, batch_tokens = batch_converter(data)
batch_lens = (batch_tokens != alphabet.padding_idx).sum(1)
#batch_tokens = batch_tokens.to(device)



In [11]:
#import torchvision.models as models
#from torch.profiler import profile, record_function, ProfilerActivity

In [12]:
#with profile(activities=[ProfilerActivity.CPU],
#        profile_memory=True, record_shapes=True) as prof:
#    model(batch_tokens)

#print(prof.key_averages().table(sort_by="self_cpu_memory_usage", row_limit=10))

In [13]:
!nvidia-smi

Sat Mar 23 09:29:17 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.147.05   Driver Version: 525.147.05   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:08:00.0  On |                  N/A |
|  0%   49C    P2    56W / 240W |    864MiB /  8192MiB |      2%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
with torch.no_grad():
    results = model(batch_tokens, repr_layers=[33], return_contacts=True)
token_representations = results["representations"][33]



In [ ]:
# Generate per-sequence representations via averaging
# NOTE: token 0 is always a beginning-of-sequence token, so the first residue is token 1.
sequence_representations = []
for i, tokens_len in enumerate(batch_lens):
    sequence_representations.append(token_representations[i, 1 : tokens_len - 1].mean(0))

# Look at the unsupervised self-attention map contact predictions
import matplotlib.pyplot as plt
for (_, seq), tokens_len, attention_contacts in zip(data, batch_lens, results["contacts"]):
    plt.matshow(attention_contacts[: tokens_len, : tokens_len])
    plt.title(seq)
    plt.show()